# Processamento de dados - Voos

Este notebook refere-se aos dados de voos coletados da ANAC.
Neste processamento ajustamos o conjuto de dados brutos com os seguintes passos:

- Leitura de dados
- Ajuste de tipos de dados 
- Filtros de premissas (exemplo: somente voos REALIZADOS serão considerados)
- Merge de datasets
- Criar novas features
- Selecionar colunas relevantes


In [1]:
import numpy as np 
import pandas as pd
import functools
import warnings
import holidays

from datetime import datetime


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
# Leitura de Dados

# VRA infos
df_voos = pd.read_parquet('df_voos.parquet')
# Aeroportos
df_aeroporto = pd.read_excel('aeroportos_BR.xlsx')
# Empresas Areas
df_cia_aerea = pd.read_csv('ANAC_operador_aereo-empresas_aereas.csv', sep=';')

In [3]:
df_voos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875274 entries, 0 to 85366
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Sigla ICAO Empresa Aérea      875274 non-null  object 
 1   Número Voo                    875274 non-null  object 
 2   Código DI                     875274 non-null  object 
 3   Código Tipo Linha             875271 non-null  object 
 4   Sigla ICAO Aeroporto Origem   875274 non-null  object 
 5   Sigla ICAO Aeroporto Destino  875274 non-null  object 
 6   Partida Prevista              844773 non-null  object 
 7   Partida Real                  834102 non-null  object 
 8   Chegada Prevista              844773 non-null  object 
 9   Chegada Real                  834081 non-null  object 
 10  Situacão Voo                  262771 non-null  object 
 11  Número de Assentos            72527 non-null   float64
 12  Situação Voo                  612503 non-null

In [4]:
df_aeroporto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   IATA        52 non-null     object
 1   ICAO        52 non-null     object
 2   AEROPORTOS  52 non-null     object
 3   CIDADE      52 non-null     object
 4   UF          52 non-null     object
 5   PAIS        52 non-null     object
dtypes: object(6)
memory usage: 2.6+ KB


In [5]:
df_cia_aerea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Razão Social              13 non-null     object
 1   ICAO IATA                 13 non-null     object
 2   CNPJ                      13 non-null     object
 3   Atividades Aéreas         13 non-null     object
 4   Endereço Sede             13 non-null     object
 5   Telefone                  13 non-null     object
 6   E-Mail                    12 non-null     object
 7   Decisão Operacional       13 non-null     object
 8   Data Decisão Operacional  13 non-null     object
 9   Validade Operacional      5 non-null      object
dtypes: object(10)
memory usage: 1.1+ KB


In [6]:
df_voos["Situacão Voo"].value_counts(dropna=False)

None             612503
REALIZADO        251801
CANCELADO         10878
NÃO INFORMADO        92
Name: Situacão Voo, dtype: int64

In [7]:
df_voos["Situação Voo"].value_counts(dropna=False)

REALIZADO        582302
None             262771
CANCELADO         29669
NÃO INFORMADO       532
Name: Situação Voo, dtype: int64

In [8]:
df_voos["Situação Voo"] = np.where(df_voos["Situação Voo"].isna(),
                                   df_voos["Situacão Voo"], 
                                   df_voos["Situação Voo"]
)

In [9]:
df_voos["Situação Voo"].value_counts(dropna=False)

REALIZADO        834103
CANCELADO         40547
NÃO INFORMADO       624
Name: Situação Voo, dtype: int64

### Regras de Negócio e Filtros

**Código DI**: Descrição do grupo DI ou Tipo de Voo. Refere-se ao agrupamento das classificações de DI em tipos de operações:

1. Regular (DIs 0, 4 ou C)
2. Não regular (DIs 1,2,5,7,8,9 ou D)
3. Improdutivo (Demais DIs)

**Código Tipo Linha**: Caractere que identifica o tipo de linha. O Tipo de Linha identifica o tipo de operação realizada no voo, podendo ser classificado em:
- N - Doméstica Mista: para operações de transporte aéreo de passageiros ou mistas, em que todos os aeródromos envolvidos estejam situados simultaneamente em território brasileiro;

- C - Doméstica Cargueira: para operações de transporte aéreo exclusivo de carga e/ou mala postal em que todos os aeródromos envolvidos estejam situados simultaneamente em território brasileiro;

- I - Internacional Mista: para operações de transporte aéreo de passageiros ou mistas, em que ao menos um dos aeródromos envolvidos esteja situado em território estrangeiro;

- G - Internacional Cargueira: para operações de transporte aéreo exclusivo de carga e/ou rede postal em que ao menos um dos aeródromos envolvidos esteja situado em território estrangeiro;

In [10]:
df_voos = df_voos[
    (df_voos["Situação Voo"] == "REALIZADO") & 
    (df_voos["Código DI"].isin(['0', '4', 'C'])) &
    (df_voos["Código Tipo Linha"] == 'N')
]

In [11]:
# Extração parâmetro de datas

df_voos["data"] = df_voos["Partida Prevista"].str[:10]

In [12]:
# Ajuste de valores para o formato de datetime 
def str_to_date(data):
    try:
        if(len(data) > 0):
            dia = int(data[0:2])
            mes = int(data[3:5])
            ano = int(data[6:10])
            hora = int(data [11:13])
            minuto = int(data[-2:])

            return datetime(ano, mes, dia, hora, minuto)
        return pd.NaT
    except:
        return pd.NaT

In [13]:
colunas_datas = ["Partida Prevista", "Partida Real", "Chegada Prevista", "Chegada Real"]

df_voos[colunas_datas] = df_voos[colunas_datas].fillna("")

In [14]:
# Conversão de tipo para datas
for coluna in colunas_datas:
    df_voos[coluna] = df_voos[coluna].map(str_to_date)

# Remove entradas inválidas para datas
df_voos = df_voos.dropna(subset=colunas_datas)

In [15]:
df_voos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 691291 entries, 515 to 83554
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Sigla ICAO Empresa Aérea      691291 non-null  object        
 1   Número Voo                    691291 non-null  object        
 2   Código DI                     691291 non-null  object        
 3   Código Tipo Linha             691291 non-null  object        
 4   Sigla ICAO Aeroporto Origem   691291 non-null  object        
 5   Sigla ICAO Aeroporto Destino  691291 non-null  object        
 6   Partida Prevista              691291 non-null  datetime64[ns]
 7   Partida Real                  691291 non-null  datetime64[ns]
 8   Chegada Prevista              691291 non-null  datetime64[ns]
 9   Chegada Real                  691291 non-null  datetime64[ns]
 10  Situacão Voo                  211713 non-null  object        
 11  Número de As

In [16]:
rename_columns = {
    'Partida Prevista': 'partida_prevista', 
    'Chegada Prevista': 'chegada_prevista',
    'Partida Real': 'partida_real',
    'Chegada Real': 'chegada_real'
}

In [17]:
df_voos = df_voos.rename(columns=rename_columns)

#### Aeroporto Info

In [18]:
# Obtem os nomes do aeroporto confomre as duas instituicoes IATA e ICAO

df_aeroporto[["Nome IATA", "Nome ICAO"]] = df_aeroporto["AEROPORTOS"].str.split('/', 1, expand=True)

In [19]:
df_voo_origem = df_voos.merge(df_aeroporto[["IATA","ICAO"]],
                              left_on = ["Sigla ICAO Aeroporto Origem"],
                              right_on = ["ICAO"],
                              how="inner"
)

In [20]:
df_voo_aeroporto = df_voo_origem.merge(df_aeroporto[["IATA","ICAO"]],
                                       left_on = ["Sigla ICAO Aeroporto Destino"],
                                       right_on = ["ICAO"],
                                       how="inner",
                                       suffixes=["_origem", "_destino"]
)

#### Companhia Aérea Info

In [21]:
# Otem o codigo IATA da empresa aerea
df_cia_aerea["ICAO_empresa"] = df_cia_aerea["ICAO IATA"].str[:3]

In [22]:
# Obtem o nome da empresa aérea
df_cia_aerea["cia_aerea"] = df_cia_aerea["Razão Social"].str.split(None, 1).str[0]

In [23]:
df_voo_infos = df_voo_aeroporto.merge(df_cia_aerea[["ICAO_empresa", "cia_aerea"]], 
                                      left_on="Sigla ICAO Empresa Aérea",
                                      right_on="ICAO_empresa", 
                                      how="inner", 
                                      validate="m:1"
)

### Features

In [24]:
# Rotas 
df_voo_infos["rota"] = df_voo_infos["IATA_origem"] + "_" + df_voo_infos["IATA_destino"]

# Tempo
df_voo_infos['partida_real'] = pd.to_datetime(df_voo_infos['partida_real'])

# duracao
df_voo_infos['duracao_prevista_voo'] = (df_voo_infos['partida_prevista'] - 
                                             df_voo_infos['chegada_prevista']).dt.total_seconds() / 60

df_voo_infos['duracao_real_voo'] = (df_voo_infos['partida_real'] - 
                                         df_voo_infos['chegada_real']
                                        ).dt.total_seconds() / 60

# calculo do atraso, com 15 min de tolerancia
df_voo_infos['atraso_chegada'] = df_voo_infos['chegada_real'] > (df_voo_infos['chegada_prevista'] + 
                                                                           pd.Timedelta(minutes =15))

df_voo_infos['atraso_partida'] = df_voo_infos['partida_real'] > (df_voo_infos['partida_prevista'] + 
                                                                           pd.Timedelta(minutes =15))


In [25]:
df_voo_infos['ano_partida'] = df_voo_infos['partida_prevista'].dt.year
df_voo_infos['mes_partida'] = df_voo_infos['partida_prevista'].dt.month
df_voo_infos['dia_partida'] = df_voo_infos['partida_prevista'].dt.day
df_voo_infos['hora_partida'] = df_voo_infos['partida_prevista'].dt.hour


In [26]:
df_voo_infos['tempo_atraso'] = df_voo_infos['partida_prevista'] - df_voo_infos["partida_real"]

In [27]:
# Definição de períodos

df_voo_infos['dia_semana'] = df_voo_infos['partida_prevista'].dt.dayofweek
df_voo_infos['fim_de_semana'] = df_voo_infos['dia_semana'].isin([5,6]).astype(int)

def periodo_dia(partida):
    horario = partida.hour
    if horario >= 6 and  horario < 12: return 'Manha'
    elif horario >= 12 and horario < 18: return 'Tarde'
    elif horario >= 18 and horario < 24: return 'Noite'
    return 'Madrugada'

df_voo_infos['turno_partida'] = df_voo_infos['partida_prevista'].map(periodo_dia)
df_voo_infos['turno_chegada'] = df_voo_infos['chegada_prevista'].map(periodo_dia)

#### Feriados

In [28]:
# Feriados Nacionais 

feriados = holidays.Brazil(years=[2022]).items()
feriados_nacionais = pd.DataFrame(feriados, columns=["data", "feriado"])
feriados_nacionais["data"] = pd.to_datetime(feriados_nacionais["data"]).dt.strftime('%d/%m/%Y')


# Feriados nacionais`
df_voo_infos = df_voo_infos.merge(feriados_nacionais, on="data", how="left", indicator=True)

In [29]:
df_voo_infos["feriado"] = np.where(df_voo_infos["_merge"] == "both",
                                   1,
                                   0
)

In [30]:
df_voo_infos.head()

,Sigla ICAO Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,Sigla ICAO Aeroporto Origem,Sigla ICAO Aeroporto Destino,partida_prevista,partida_real,chegada_prevista,chegada_real,Situacão Voo,Número de Assentos,Situação Voo,Unnamed: 11,data,IATA_origem,ICAO_origem,IATA_destino,ICAO_destino,ICAO_empresa,cia_aerea,rota,duracao_prevista_voo,duracao_real_voo,atraso_chegada,atraso_partida,ano_partida,mes_partida,dia_partida,hora_partida,tempo_atraso,dia_semana,fim_de_semana,turno_partida,turno_chegada,feriado,_merge
0,AZU,2451,0,N,SBRF,SBPA,2022-01-01 17:45:00,2022-01-01 17:41:00,2022-01-01 22:05:00,2022-01-01 21:49:00,REALIZADO,NaN,REALIZADO,NaN,01/01/2022,REC,SBRF,POA,SBPA,AZU,AZUL,REC_POA,-260.0,-248.0,False,False,2022,1,1,17,0 days 00:04:00,5,1,Tarde,Noite,1,both
1,AZU,2987,0,N,SBRF,SBPA,2022-01-02 17:25:00,2022-01-02 17:23:00,2022-01-02 21:50:00,2022-01-02 21:36:00,REALIZADO,NaN,REALIZADO,NaN,02/01/2022,REC,SBRF,POA,SBPA,AZU,AZUL,REC_POA,-265.0,-253.0,False,False,2022,1,2,17,0 days 00:02:00,6,1,Tarde,Noite,0,left_only
2,AZU,2638,0,N,SBRF,SBPA,2022-01-03 02:25:00,2022-01-03 02:22:00,2022-01-03 06:45:00,2022-01-03 06:30:00,REALIZADO,NaN,REALIZADO,NaN,03/01/2022,REC,SBRF,POA,SBPA,AZU,AZUL,REC_POA,-260.0,-248.0,False,False,2022,1,3,2,0 days 00:03:00,0,0,Madrugada,Manha,0,left_only
3,AZU,2987,0,N,SBRF,SBPA,2022-01-03 17:25:00,2022-01-03 17:22:00,2022-01-03 21:50:00,2022-01-03 21:29:00,REALIZADO,NaN,REALIZADO,NaN,03/01/2022,REC,SBRF,POA,SBPA,AZU,AZUL,REC_POA,-265.0,-247.0,False,False,2022,1,3,17,0 days 00:03:00,0,0,Tarde,Noite,0,left_only
4,AZU,2638,0,N,SBRF,SBPA,2022-01-04 02:25:00,2022-01-04 02:20:00,2022-01-04 06:45:00,2022-01-04 06:29:00,REALIZADO,NaN,REALIZADO,NaN,04/01/2022,REC,SBRF,POA,SBPA,AZU,AZUL,REC_POA,-260.0,-249.0,False,False,2022,1,4,2,0 days 00:05:00,1,0,Madrugada,Manha,0,left_only


In [31]:
colunas = [
    'IATA_origem', 'ICAO_origem', 'IATA_destino', 'ICAO_destino', 'data',
    'partida_prevista', 'partida_real', 'chegada_prevista', 'chegada_real','cia_aerea',
    'rota', 'duracao_prevista_voo','duracao_real_voo', 'atraso_chegada', 'atraso_partida',
    'ano_partida','mes_partida', 'dia_partida', 'hora_partida', 'tempo_atraso',
    'dia_semana', 'fim_de_semana', 'turno_partida', 'turno_chegada', 'feriado'
]

In [34]:
df_voo_infos_final = df_voo_infos[colunas]

In [35]:
df_voo_infos_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 612435 entries, 0 to 612434
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   IATA_origem           612435 non-null  object         
 1   ICAO_origem           612435 non-null  object         
 2   IATA_destino          612435 non-null  object         
 3   ICAO_destino          612435 non-null  object         
 4   data                  612435 non-null  object         
 5   partida_prevista      612435 non-null  datetime64[ns] 
 6   partida_real          612435 non-null  datetime64[ns] 
 7   chegada_prevista      612435 non-null  datetime64[ns] 
 8   chegada_real          612435 non-null  datetime64[ns] 
 9   cia_aerea             612435 non-null  object         
 10  rota                  612435 non-null  object         
 11  duracao_prevista_voo  612435 non-null  float64        
 12  duracao_real_voo      612435 non-null  float

In [36]:
df_voo_infos_final.to_parquet('info_voos.parquet')